In [2]:
! pip3 install nltk
! pip3 install Sastrawi
! pip install googletrans

# Import Library
# text preprocessing
import pandas as pd
import numpy as np
import csv
import requests
import io
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re # regular expression
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory # stemming indonesian language
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.corpus import stopwords
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer # to create Bag of words
from sklearn.feature_extraction.text import TfidfVectorizer # tfid Vector 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score # confussion matrix
from sklearn.preprocessing import LabelEncoder # to convert classes to number 
from sklearn.model_selection import train_test_split  # for splitting data 
from sklearn.metrics import accuracy_score # to calculate accuracy
from sklearn.pipeline import Pipeline

You should consider upgrading via the 'c:\users\yusrifa deta kirana\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\yusrifa deta kirana\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


[nltk_data] Downloading package punkt to C:\Users\Yusrifa Deta
[nltk_data]     Kirana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Yusrifa Deta
[nltk_data]     Kirana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
dataset = pd.read_csv(r'before_preprocessing.csv')

In [4]:
dataset.head()

,review_id,review_text,price,packaging,product,aroma
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1


In [5]:
dataset.isnull().sum()

review_id      0
review_text    0
price          0
packaging      0
product        1
aroma          0
dtype: int64

In [6]:
dataset = dataset.fillna(method="ffill")
dataset.isnull().sum()

review_id      0
review_text    0
price          0
packaging      0
product        0
aroma          0
dtype: int64

In [7]:
dataset.head()

,review_id,review_text,price,packaging,product,aroma
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1


In [8]:
# casefolding 

def clean(dataset):
  temp_text = []

  for txt in dataset:
    # removal of @name[mention]
    txt = re.sub(r"(?:\@|https?\://)\S+", "", txt)

    # removal of links[https://blabala.com]
    # tw = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", tw)
    txt = re.sub(r"http\S+", "", txt)

    # removal of new line
    txt = re.sub('\n', '', txt)

    # removal of RT
    txt = re.sub('RT', '', txt)

    # removal of punctuations and numbers
    txt = re.sub("[^a-zA-Z^']", " ", txt)
    txt = re.sub(" {2,}", " ", txt)

    # remove leading and trailing whitespace
    txt = txt.strip()

    # remove whitespace with a single space
    txt = re.sub(r'\s+', ' ', txt)

    # convert text to Lowercase
    text = txt.lower();
    temp_text.append(txt)
  return temp_text

In [9]:
dataset['Clean'] = clean(dataset['review_text'])
dataset.head()

,review_id,review_text,price,packaging,product,aroma,Clean
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0,sunscreen termahal yang pernah gue beli ini ka...
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0,ini enak banget dipakainya enteng banget diwaj...
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1,the description is quite right produk ini eman...
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0,bisa untuk wajah dan badan dengan harga yang s...
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1,saya beli produk ini karena suka banget wangin...


In [10]:
#TOKENISASI
def token(dataset):
  return dataset.apply(nltk.word_tokenize)

In [11]:
dataset['Clean'] = token(dataset['Clean'])
dataset.head()


,review_id,review_text,price,packaging,product,aroma,Clean
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0,"[sunscreen, termahal, yang, pernah, gue, beli,..."
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0,"[ini, enak, banget, dipakainya, enteng, banget..."
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1,"[the, description, is, quite, right, produk, i..."
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0,"[bisa, untuk, wajah, dan, badan, dengan, harga..."
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1,"[saya, beli, produk, ini, karena, suka, banget..."


In [74]:
#def stop_words(dataset) :
 # stop_words = set(stopwords.words('indonesian'))
 # return dataset.apply(lambda x: [item for item in x if item not in stop_words])

#dataset['Clean'] = stop_words(dataset['Clean'])
#dataset.head()

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer, CountVectorizer
tfidf_transformer = TfidfTransformer()
tfidf_vectorizer = TfidfVectorizer()
count_vectorize = CountVectorizer()

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'Pusing satu kali',
    'Pusing dua kali',
    'Pusing tiga kali',
    'Pusing lagi']

vectorizer = TfidfVectorizer()

# contoh
a = vectorizer.fit_transform(corpus)
a.toarray()


array([[0.        , 0.49248889, 0.        , 0.40264194, 0.77157901,
        0.        ],
       [0.77157901, 0.49248889, 0.        , 0.40264194, 0.        ,
        0.        ],
       [0.        , 0.49248889, 0.        , 0.40264194, 0.        ,
        0.77157901],
       [0.        , 0.        , 0.88654763, 0.46263733, 0.        ,
        0.        ]])

In [14]:
a = dataset['review_text']
# Count TF_IDF Vectorizer
count_vectorizer = CountVectorizer()
count_vector = count_vectorizer.fit_transform(dataset['review_text'])
count_vector.shape

(3960, 11919)

In [15]:
a = dataset['review_text']
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer = tfidf_vectorizer.fit_transform(a)
print(tfidf_vectorizer)


  (0, 10508)	0.21755947863811115
  (0, 10910)	0.1918674438131206
  (0, 11845)	0.04627303732722482
  (0, 8536)	0.09692855006425057
  (0, 4104)	0.29197011723583527
  (0, 1146)	0.0725581512259172
  (0, 4640)	0.10104005098570525
  (0, 5090)	0.13978095002176325
  (0, 1855)	0.11656957507439628
  (0, 4717)	0.18071324567985922
  (0, 11792)	0.11608443100934708
  (0, 10065)	0.07028628025894527
  (0, 614)	0.10551113714137114
  (0, 11744)	0.13289626263283422
  (0, 2493)	0.1413066468813367
  (0, 1868)	0.1533423673625266
  (0, 417)	0.1700491709924254
  (0, 10478)	0.17890012247991563
  (0, 9363)	0.06437169788817153
  (0, 10502)	0.18960875722150158
  (0, 4369)	0.17890012247991563
  (0, 10304)	0.10965967531992081
  (0, 7670)	0.1871194898606684
  (0, 6891)	0.14890682313598788
  (0, 11722)	0.13065996280800182
  :	:
  (3959, 2329)	0.09800820655599626
  (3959, 9007)	0.10589448419949808
  (3959, 1717)	0.11420943446195812
  (3959, 9382)	0.13581543808084823
  (3959, 7898)	0.09588989617557801
  (3959, 5510)	0.

In [16]:
tfidf_vectorizer.shape

(3960, 11919)

In [17]:
x = tfidf_vectorizer.copy()

In [18]:
y = np.array(dataset[['price','packaging','product','aroma']])

In [19]:
print(y)

[[-1.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  1.]
 ...
 [ 0.  0. -1. -1.]
 [ 0.  0.  1.  0.]
 [ 0.  0. -1.  0.]]


In [20]:
x.shape, y.shape

((3960, 11919), (3960, 4))

In [21]:
from sklearn.preprocessing import MultiLabelBinarizer
Y = MultiLabelBinarizer().fit_transform(y)

In [22]:
from sklearn.model_selection import train_test_split
x_train, x_test, Y_train, Y_test = train_test_split(x,Y, test_size=0.3, random_state=0)

In [74]:
x_train.shape, x_test.shape

((2772, 11919), (1188, 11919))

In [123]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor


knn = KNeighborsClassifier(n_neighbors=3)
## Fit the model on the training data.
classifier = MultiOutputClassifier(knn, n_jobs=-1)
classifier.fit(x_train, Y_train)
## See how the model performs on the test data.
predictions = classifier.predict(x_test)

In [118]:
knn_skor=classifier.score(x_test,np.array(Y_test))
print(knn_skor * 100 ,'%')

57.07070707070707 %


In [134]:
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import recall_score
from sklearn.metrics import  multilabel_confusion_matrix

multilabel = multilabel_confusion_matrix(Y_test,predictions)
f1_score = metrics.f1_score(Y_test,predictions, average='micro', labels=np.unique(predictions))
precision_score = metrics.precision_score(Y_test,predictions, average='micro', labels=np.unique(predictions))
recall_score = metrics.recall_score(Y_test,predictions, average='micro', labels=np.unique(predictions))
        
print('akurasi f1 score        :', f1_score* 100 ,'%')
print('akurasi precision score :', precision_score* 100 ,'%')
print('akurasi recall score    :', recall_score* 100 ,'%')
#print (multilabel)

akurasi f1 score        : 86.42506142506143 %
akurasi precision score : 87.9375 %
akurasi recall score    : 84.96376811594203 %


In [130]:
from sklearn.metrics import accuracy_score, classification_report, multilabel_confusion_matrix

In [131]:
y_pred = classifier.predict(x_test)

In [136]:
# classification_report
#multilabel = multilabel_confusion_matrix(Y_test,predictions)
print(multilabel_confusion_matrix(Y_test,predictions))


[[[ 527  175]
  [ 248  238]]

 [[   0   18]
  [   1 1169]]

 [[  39  179]
  [  51  919]]]
